# Tratamento inicial dos dados

Objetivo:
1. Carregar os dados;
2. Selecionar as colunas que serão lidas;
3. Fazer uma amostragem aleatória de 800.000 dados (~20% do dataset);
4. Salvar em um novo ``.csv`` que seja mais leve

## Seleção de variáveis e amostragem dos dados

**NOTA SOBRE O RESULTADO DA LLM:** O algoritmo gerado para selecionar e amostrar os dados foi bastante satisfatório, a não ser por um pequeno erro lógico no final: na fase ``# junta tudo``, o código tenta tirar novamente 800_000 amostras de uma lista que já tem (teoricamente) 800_000 amostras, então essa amostragem não faz sentido. Na prática, ainda, o código provavelmente vai gerar um erro, já que nosso banco não tem exatamente 4M registros e o resultado terá um pouco menos que 800k amostras.

In [ ]:
""""
GERA O ARQUIVO SAMPLE, NÃO RODAR SE O ARQUIVO JÁ ESTIVER CRIADO
"""

import pandas as pd
import numpy as np
from pathlib import Path

# Configurações
DATA_PATH = Path().resolve() / 'data'
ARQUIVO_ORIGEM_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023.csv'
ARQUIVO_DESTINO_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023_sample.csv'

colunas_desejadas = [
    'NU_INSCRICAO', 'TP_FAIXA_ETARIA', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
    'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU', 'TP_ESCOLA', 'TP_ENSINO',
    'IN_TREINEIRO', 'CO_MUNICIPIO_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC', 'CO_MUNICIPIO_PROVA', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
    'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC',
    'CO_PROVA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'TP_LINGUA',
    'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q006', 'Q025'
]

# Parâmetros
tamanho_amostra_final = 800_000
tamanho_arquivo_total = 4_000_000  # Aproximadamente
frac_amostragem = tamanho_amostra_final / tamanho_arquivo_total

# Leitura em chunks
chunk_size = 500_000
amostras = []

for chunk in pd.read_csv(ARQUIVO_ORIGEM_PATH, usecols=colunas_desejadas, chunksize=chunk_size, sep=';', encoding='latin1'):
    amostra_chunk = chunk.sample(frac=frac_amostragem, random_state=553)
    amostras.append(amostra_chunk)

# Junta tudo
# amostra_final = pd.concat(amostras).sample(n=tamanho_amostra_final, random_state=42) # resultado da LLM
amostra_final = pd.concat(amostras)

# Salva no CSV
amostra_final.to_csv(ARQUIVO_DESTINO_PATH, index=False)

print(f"Amostra salva em {ARQUIVO_DESTINO_PATH}")

## Leitura eficiente do dataframe amostrado

**NOTA SOBRE O RESULTADO DA LLM:** Constroi um dicionário de dtypes de forma inteligente, mas: i) faz um ``col for col ...`` desnecessariamente em ``colunas_float``. 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Configurações
DATA_PATH = Path().resolve() / 'data'
ARQUIVO_AMOSTRA_PATH = DATA_PATH / 'raw' / 'microdados_enem_2023_sample.csv'

# Definição dos tipos
colunas_float = [
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
    'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4',
    'NU_NOTA_COMP5', 'NU_NOTA_REDACAO'
]

colunas_string = [
    'NU_INSCRICAO', 'CO_MUNICIPIO_ESC', 'CO_MUNICIPIO_PROVA'
]

# Captura os nomes das colunas
colunas = pd.read_csv(ARQUIVO_AMOSTRA_PATH, nrows=0, encoding='latin1').columns.tolist()

# Preparar o dicionário de tipos
dtypes = {}
for col in colunas:
    if col in colunas_float:
        dtypes[col] = 'float32'
    elif col in colunas_string:
        dtypes[col] = 'string'
    else:
        dtypes[col] = 'category'

# Leitura com tipos otimizados
df = pd.read_csv(ARQUIVO_AMOSTRA_PATH, dtype=dtypes, encoding='latin1')

# Informações para conferência
print("="*40)
print(f"Memória usada: {df.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")
print(f"Quantidade de colunas: {df.shape[1]}")
print(f"Quantidade de linhas: {df.shape[0]}")
print("="*40)

# Tipos de dados
print("\nTipos de dados por coluna:")
print(df.dtypes)

Memória usada: 180.46 MB
Quantidade de colunas: 40
Quantidade de linhas: 786791

Tipos de dados por coluna:
NU_INSCRICAO              string[python]
TP_FAIXA_ETARIA                 category
TP_SEXO                         category
TP_ESTADO_CIVIL                 category
TP_COR_RACA                     category
TP_NACIONALIDADE                category
TP_ST_CONCLUSAO                 category
TP_ANO_CONCLUIU                 category
TP_ESCOLA                       category
TP_ENSINO                       category
IN_TREINEIRO                    category
CO_MUNICIPIO_ESC          string[python]
TP_DEPENDENCIA_ADM_ESC          category
TP_LOCALIZACAO_ESC              category
TP_SIT_FUNC_ESC                 category
CO_MUNICIPIO_PROVA        string[python]
TP_PRESENCA_CN                  category
TP_PRESENCA_CH                  category
TP_PRESENCA_LC                  category
TP_PRESENCA_MT                  category
CO_PROVA_CN                     category
CO_PROVA_CH                    

## Exploratory Data Analysis

In [2]:
# Checagem de valores nulos
print("\nAno que concluiu:")
nulos = df.isnull().sum()
percentual_nulos = (nulos / len(df)) * 100


tp_escola = df['TP_ANO_CONCLUIU'].value_counts(dropna=False)
tp_escola_pct = df['TP_ANO_CONCLUIU'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


Ano que concluiu:
                 Frequência  Percentual (%)
TP_ANO_CONCLUIU                            
0                    448386           56.99
1                     83779           10.65
2                     52843            6.72
17                    33560            4.27
3                     29923            3.80
4                     26964            3.43
5                     20932            2.66
6                     17226            2.19
7                     13114            1.67
8                     11043            1.40
9                      9319            1.18
10                     8082            1.03
11                     7119            0.90
12                     5774            0.73
13                     5540            0.70
14                     4823            0.61
15                     4222            0.54
16                     4142            0.53


In [3]:
# Checagem de valores nulos
print("\nConclusão:")
nulos = df.isnull().sum()
percentual_nulos = (nulos / len(df)) * 100


tp_escola = df['TP_ST_CONCLUSAO'].value_counts(dropna=False)
tp_escola_pct = df['TP_ST_CONCLUSAO'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


Conclusão:
                 Frequência  Percentual (%)
TP_ST_CONCLUSAO                            
1                    379427           48.22
2                    280154           35.61
3                    123745           15.73
4                      3465            0.44


In [ ]:

filtro = df[df['TP_ANO_CONCLUIU'] == '0']
grupo = filtro.groupby('TP_ST_CONCLUSAO', dropna=False)['NU_INSCRICAO']

resultado = pd.DataFrame({
    'Total': grupo.count()
})

resultado['Percentual (%)'] = (resultado['Total'] / resultado['Total'].sum() * 100).round(2)

print(resultado)
print(resultado['Total'].sum())

                  Total  Percentual (%)
TP_ST_CONCLUSAO                        
1                 41022            9.15
2                280154           62.48
3                123745           27.60
4                  3465            0.77
448386


/tmp/ipykernel_42591/3863778741.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grupo = filtro.groupby('TP_ST_CONCLUSAO')['NU_INSCRICAO']


In [18]:
df[df['TP_ST_CONCLUSAO'] != '1'].count()

NU_INSCRICAO              407364
TP_FAIXA_ETARIA           407364
TP_SEXO                   407364
TP_ESTADO_CIVIL           407364
TP_COR_RACA               407364
TP_NACIONALIDADE          407364
TP_ST_CONCLUSAO           407364
TP_ANO_CONCLUIU           407364
TP_ESCOLA                 407364
TP_ENSINO                 267749
IN_TREINEIRO              407364
CO_MUNICIPIO_ESC          191881
TP_DEPENDENCIA_ADM_ESC    191881
TP_LOCALIZACAO_ESC        191881
TP_SIT_FUNC_ESC           191881
CO_MUNICIPIO_PROVA        407364
TP_PRESENCA_CN            407364
TP_PRESENCA_CH            407364
TP_PRESENCA_LC            407364
TP_PRESENCA_MT            407364
CO_PROVA_CN               316098
CO_PROVA_CH               331719
CO_PROVA_LC               331719
CO_PROVA_MT               316098
NU_NOTA_CN                316098
NU_NOTA_CH                331719
NU_NOTA_LC                331719
NU_NOTA_MT                316098
TP_LINGUA                 407364
TP_STATUS_REDACAO         331719
NU_NOTA_CO

In [19]:
tirou0 = df.NU_NOTA_CH[df.NU_NOTA_CH == 0].count()
tirou_mais_q_0 = df.NU_NOTA_CH[df.NU_NOTA_CH > 0].count()
total = df.NU_NOTA_CH.count()

print(tirou0, '\t', 100.0*tirou0/total)
print(tirou_mais_q_0, '\t', 100.0*tirou_mais_q_0/total)
print(total, '\t', 100.0*total/total)

1107 	 0.19608746173903185
563437 	 99.80391253826097
564544 	 100.0


In [20]:
df[df.NU_NOTA_CH == 0].groupby('TP_PRESENCA_CH').count()

/tmp/ipykernel_42591/4160640221.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[df.NU_NOTA_CH == 0].groupby('TP_PRESENCA_CH').count()


,NU_INSCRICAO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,...,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q025
TP_PRESENCA_CH,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1107,1107,1107,1107,1107,1107,1107,1107,1107,377,...,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Tabela cruzada entre duas colunas categóricas
tabela = pd.crosstab(df['TP_ANO_CONCLUIU'], df['TP_ST_CONCLUSAO'][df['TP_ST_CONCLUSAO'] == 0])

# Heatmap
sns.heatmap(tabela, annot=True, fmt='d', cmap='Blues')
plt.xlabel('COLUNA2')
plt.ylabel('COLUNA1')
plt.title('Heatmap entre COLUNA1 e COLUNA2')
plt.show()

: 

In [3]:
# Checagem de valores nulos
print("\nValores nulos por coluna:")
nulos = df.isnull().sum()
percentual_nulos = (nulos / len(df)) * 100

# Criando um dataframe com os valores absolutos e percentuais
df_nulos = pd.DataFrame({
    'Valores Nulos': nulos,
    'Percentual Nulos (%)': percentual_nulos
})

# Exibindo o dataframe de nulos
print(df_nulos)


Valores nulos por coluna:
                        Valores Nulos  Percentual Nulos (%)
NU_INSCRICAO                        0              0.000000
TP_FAIXA_ETARIA                     0              0.000000
TP_SEXO                             0              0.000000
TP_ESTADO_CIVIL                     0              0.000000
TP_COR_RACA                         0              0.000000
TP_NACIONALIDADE                    0              0.000000
TP_ST_CONCLUSAO                     0              0.000000
TP_ANO_CONCLUIU                     0              0.000000
TP_ESCOLA                           0              0.000000
TP_ENSINO                      519042             65.969489
IN_TREINEIRO                        0              0.000000
CO_MUNICIPIO_ESC               594910             75.612202
TP_DEPENDENCIA_ADM_ESC         594910             75.612202
TP_LOCALIZACAO_ESC             594910             75.612202
TP_SIT_FUNC_ESC                594910             75.612202
CO_MUNICIPIO_

In [ ]:
# Checagem de valores nulos
print("\nTipo de escola:")

tp_escola = df['TP_ESCOLA'].value_counts(dropna=False)
tp_escola_pct = df['TP_ESCOLA'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


Tipo de escola:
           Frequência  Percentual (%)
TP_ESCOLA                            
1              506638           64.39
2              233307           29.65
3               46846            5.95


In [ ]:
median = df.NU_NOTA_MT.median() 
acima_da_mediana = df.NU_NOTA_MT[df.NU_NOTA_MT > median]

df.groupby().agg({'NU_NOTA_MT': 'mean', 'SEXO': 'count'})

: 

In [6]:
# Checagem de valores nulos
print("\nTipo de ensino:")

tp_escola = df['TP_ENSINO'].value_counts(dropna=False)
tp_escola_pct = df['TP_ENSINO'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


Tipo de ensino:
           Frequência  Percentual (%)
TP_ENSINO                            
NaN            519042           65.97
1.0            266392           33.86
2.0              1357            0.17


In [5]:
# Checagem de valores nulos
print("\nPresença Mat:")

tp_escola = df['TP_PRESENCA_MT'].value_counts(dropna=False)
tp_escola_pct = df['TP_PRESENCA_MT'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


Presença Mat:
                Frequência  Percentual (%)
TP_PRESENCA_MT                            
1                   538319           68.42
0                   247988           31.52
2                      484            0.06


In [ ]:
# Checagem de valores nulos
print("\n:")

tp_escola = df['TP_PRESENCA_MT'].value_counts(dropna=False)
tp_escola_pct = df['TP_PRESENCA_MT'].value_counts(normalize=True, dropna=False) * 100

resultado = pd.DataFrame({
    'Frequência': tp_escola,
    'Percentual (%)': tp_escola_pct.round(2)
})

print(resultado)


: 